Se importan las librerias necesarias para el analisis de datos y predicción

link:"https://www.kaggle.com/datasets/robjan/ph-recognition"

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import r2_score, mean_squared_error, accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV, train_test_split, RandomizedSearchCV, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

Se procede a leer los datos del archivo csv y verificar el estado de los datos importados

In [3]:
df_ph = pd.read_csv('pH Data.csv')
df_ph_sh = df_ph.sample(frac=1, random_state=7).reset_index(drop=True)

display(df_ph.head(5))
print(df_ph.info())
print(df_ph.describe())

,blue,green,red,label
0,36,27,231,0
1,36,84,250,1
2,37,164,255,2
3,22,205,255,3
4,38,223,221,4


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 653 entries, 0 to 652
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   blue    653 non-null    int64
 1   green   653 non-null    int64
 2   red     653 non-null    int64
 3   label   653 non-null    int64
dtypes: int64(4)
memory usage: 20.5 KB
None
             blue       green         red       label
count  653.000000  653.000000  653.000000  653.000000
mean    89.290965  130.094946  120.655436    7.055130
std     74.974850   72.487085   89.581264    4.287951
min      0.000000    0.000000    0.000000    0.000000
25%     22.000000   68.000000   52.000000    3.000000
50%     67.000000  146.000000   90.000000    7.000000
75%    162.000000  189.000000  224.000000   11.000000
max    255.000000  255.000000  255.000000   14.000000


No es necesario realizar limpieza de datos ya que existen todos los datos y no hay valores atípicos.
Se procede a dividir los datos en conjuntos de entrenamiento y de testeo

In [4]:
X = df_ph_sh.drop('label', axis=1)
y = df_ph_sh['label']

X_tr, X_ts, y_tr, y_ts = train_test_split(X, y, test_size = 0.21, random_state=4)

Se realiza un Pipelane para estandarizar los datos, y se aplica bosques aleatorios para el modelamiento de datos.
Se realiza un random search para encontrar de manera aleatoria los mejores parametros de operacion del modelo


In [5]:
RFC_Pipeline = Pipeline([
                    ('Scale', StandardScaler()),
                    ('RFC', RandomForestClassifier(random_state=7, warm_start=True))
                ])

param_grid = {
                    'RFC__n_estimators': [38,39,40,41,42],
                    'RFC__max_depth': [3,4,5,6]
                } 

randSCV = RandomizedSearchCV(estimator=RFC_Pipeline, param_distributions=param_grid, n_iter=12, scoring='accuracy', cv=10, refit=True, return_train_score=True)

randSCV.fit(X_tr, y_tr)

#print(gridSCV.cv_results_)
print("Complete details of the model:  ",randSCV.best_estimator_)
print("Best scorer:                    ",randSCV.best_score_)
print("Best Hyperparameters choosen:   ",randSCV.best_params_)
print("Best Index:                     ",randSCV.best_index_)
print("Scorer:                         ",randSCV.scorer_)
print("number of splits:               ",randSCV.n_splits_)
print("refit_time:                     ",randSCV.refit_time_)


BMRand_RFC_Pipeline = randSCV.best_estimator_ # BM: best model

scores = cross_val_score(estimator=BMRand_RFC_Pipeline, X=X_tr, y=y_tr, cv=15, scoring='accuracy') 

y_pred_RandBMRFC = BMRand_RFC_Pipeline.predict(X_ts)

print('++++++++++++++++Scores: ')
for i in scores:
    print(i)
print('\n')  
print("Mean:", scores.mean())
print("Standard deviation:", scores.std())
print('\n')
print('Score on the TEST set:          ', accuracy_score(y_ts, y_pred_RandBMRFC))

Complete details of the model:   Pipeline(steps=[('Scale', StandardScaler()),
                ('RFC',
                 RandomForestClassifier(max_depth=6, n_estimators=39,
                                        random_state=7, warm_start=True))])
Best scorer:                     0.7570135746606335
Best Hyperparameters choosen:    {'RFC__n_estimators': 39, 'RFC__max_depth': 6}
Best Index:                      2
Scorer:                          make_scorer(accuracy_score)
number of splits:                10
refit_time:                      0.04941964149475098
++++++++++++++++Scores: 
0.8571428571428571
0.8
0.8
0.8571428571428571
0.7714285714285715
0.7058823529411765
0.8529411764705882
0.7647058823529411
0.5882352941176471
0.8529411764705882
0.7647058823529411
0.7647058823529411
0.7647058823529411
0.7352941176470589
0.5588235294117647


Mean: 0.7625770308123249
Standard deviation: 0.08678838557053661


Score on the TEST set:           0.717391304347826


Se obtuvo un valor de 71,74% de presición de los valores de testeo en su predicción de pH.
A continuación se propone un espacio para insertar datos de azul verde y rojo y predecir pH

In [7]:
azul = 88
verde = 127
rojo = 231
eval = [azul, verde, rojo]
eval = np.array(eval)
resultado =BMRand_RFC_Pipeline.predict(eval.reshape(1,-1))
print('el valor de pH para la nueva muestra es de: .......  {}'.format(resultado))

el valor de pH para la nueva muestra es de: .......  [2]


/home/yordan/.local/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
